# Convolutional Neural Networks For Image Recognition

## Setup and Imports

In [1]:
# Setup
from __future__ import division, print_function, unicode_literals

# Imports
import numpy as np
import numpy.random as rnd
import os

# To make make consistent across code blocks
rnd.seed(42)

# Ploting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Preprocessing functions
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Saving Parameters
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "data", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

**Import Keras**

In [2]:
import keras as kr

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


# Convolutional Neural Network

## Import Dataset

In [3]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

In [4]:
def to_normalized(X):
    X = (X-128)/128
    return X
    
def to_one_hot(X):
    n_values = np.max(X) + 1
    y = np.eye(n_values)[X[:,0]]
    return y

In [5]:
x_train = to_normalized(x_train)
x_test = to_normalized(x_test)

In [6]:
y_train = to_one_hot(y_train)
y_test  = to_one_hot(y_test)

In [7]:
def get_val_set(X, y, percent):
    X_s, y_s = shuffle(X, y, random_state=0)
    
    index = int(percent*len(y))
    X_val = X_s[:index]
    y_val = y_s[:index]
    X_out = X_s[index:]
    y_out = y_s[index:]
    
    return X_out, y_out, X_val, y_val

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.1, random_state=0)

## Image Preprocessing

In [9]:
def clip_n_flip(X, Y):
    Y_conc = np.concatenate([Y, Y, Y, Y, Y, Y, Y, Y, Y, Y], axis=0)
    X_a = X[:,:28,:28]
    X_b = X[:,4:,:28]
    X_c = X[:,:28,4:]
    X_d = X[:,4:,4:]
    X_e = X[:,2:30,2:30]
    X_clip = np.concatenate([X_a,X_b,X_c,X_d,X_e], axis=0)
    X_flip = np.concatenate([X_clip, np.fliplr(X_clip)], axis=0)
    return X_flip, Y_conc

In [10]:
def clip_center(X):
    return X[:,2:30,2:30]

In [11]:
x_train, y_train = clip_n_flip(x_train, y_train)
X_out, Y_out = shuffle(x_train, y_train, random_state=1)

In [12]:
x_val = clip_center(x_val)
x_test = clip_center(x_train)

## Coarse Training

** Defining Structure**

In [13]:
def indep_layers(X, set_index):
    s = '__indep_layer_'
    net = Conv2D(32, (2, 2), strides=(1, 1), padding='same', activation='elu', name=set_index+s+str(2))(X)
    net = Dropout(0.50)(net)
    net = Conv2D(32, (2, 2), strides=(1, 1), padding='same', activation='elu', name=set_index+s+str(3))(X)
    net = Flatten()(net)

    net = Dense(64, activation='elu', name=set_index+s+str(4))(net)
    net = Dropout(0.50)(net)
    net = Dense(64, activation='elu', name=set_index+s+str(5))(net)
    net = Dropout(0.50)(net)

    
    return net

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout, Multiply, Add, Concatenate, Dot, RepeatVector, Reshape
from keras.optimizers import SGD
from keras.models import Model

coarse_categories = 20
fine_categories = 100
sigma = .01

input_img = Input(shape=(28, 28, 3), dtype='float32', name='main_input')

## SHARED LAYERS
net_shared = Conv2D(8, (4, 4), strides=(1, 1), padding='same', activation='elu')(input_img)
net_shared = MaxPooling2D((2, 2), padding='valid')(net_shared)

net_shared = Conv2D(16, (4, 4), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = MaxPooling2D((2, 2), padding='valid')(net_shared)

net_shared = Conv2D(32, (4, 4), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = Dropout(0.50)(net_shared)

# COARSE CLASSIFIER
# Get the coarse predictions from fine
net = indep_layers(net_shared, '0')
output_c = Dense(100, activation='softmax')(net)

base_model = Model(inputs=input_img, outputs=output_c)

base_model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
num_epochs = 2
base_model.fit(x_train, y_train, initial_epoch=0, epochs=num_epochs, batch_size=256, validation_data=(x_val, y_val))
base_model.save_weights('data/models/base_model_'+str(num_epochs))

Train on 450000 samples, validate on 5000 samples
Epoch 1/2
128000/450000 [=======>......................] - ETA: 447s - loss: 4.4470 - acc: 0.0311

## Fine-Tuning

**Define New Thresholding Layer**

In [ ]:
from __future__ import absolute_import

from keras.layers import initializers, regularizers, constraints
from keras.engine.topology import Layer, InputSpec
from keras import backend as K
from keras.legacy import interfaces

class Threshold(Layer):
    """Thresholded Rectified Linear Unit.
    This layer takes in a set of coarse probabilities and an
    index pointing to the relevant coarse probability
    
    It then thresholds that probability following:
    `f(x) = 1 for x > theta`,
    `f(x) = 0 otherwise`.
    
    Finally it multiplies that thresholded value across the 
    full input layer.
    
    # Input shape
        Arbitrary 4-D Tensor
    # Output
        An tensor with the same shape as the input but with values
        that are either all 1s or all 0s
    # Arguments
        theta: float >= 0. Threshold value.
    """

    def __init__(self, prob, index, theta=.5, **kwargs):
        super(Threshold, self).__init__(**kwargs)
        self.theta = K.cast_to_floatx(theta)
        self.index = index
        self.prob = prob

    def call(self, inputs, mask=None):
        # Get Shape of Input
        shape = inputs.get_shape().as_list()
        # Pull out coarse probability as specified by self.prob
        X = self.prob[:,self.index:self.index+1]
        # Threshold probability
        X = K.cast(K.greater(X, self.theta), K.floatx())
        # Repeat and Reshape thresholded probability to match input
        X = K.tile(X, [1, shape[1]*shape[2]*shape[3]])
        X = K.reshape(X, [-1,shape[1],shape[2],shape[3]])
        # Multiply input with thresholded probability
        X = Multiply()([X,inputs])
        return X

    def get_config(self):
        config = {'theta': float(self.theta)}
        base_config = super(Threshold, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

**Add Fine Layers**

In [ ]:
 def calc_prob_ave(X, c2k):
    X = RepeatVector(1)(X)
    X = Dot(axes=2)([ck,X])
    Y = Reshape([20])(X)
    X = Dot(axes=1)([ck,X])
    X = Reshape([100])(X)
    return X, Y


In [ ]:
ck = Input(shape=(20, 100), dtype='float32', name='C2K')
prob_ave, coarse_prob = calc_prob_ave(output_c, ck)

for i in range(20):
    #thresholded = Threshold(coarse_prob,i)(net_shared)
    net = indep_layers(net_shared, str(i+1))
    fine_output = Dense(5, activation='softmax')(net)
    
    if i==0: output_f = fine_output
    else: output_f = Concatenate(axis=1)([output_f, fine_output])


weighted = Multiply()([output_f, prob_ave])

**Compile new network**

In [ ]:
model = Model(inputs=[base_model.input, ck], 
              outputs=output_f)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Load Coarse-To-Fine Mappings**

In [ ]:
# Get Coarse to Fine Mappings
C2K = np.loadtxt('C2K.txt', dtype=int)
C2K_dot = np.zeros((20,100))
for i in range(len(C2K)):
    C2K_dot[C2K[i], i] = 1

#np.set_printoptions(threshold=np.inf)
#print(C2K_dot)

In [ ]:
C2K_train = np.zeros((len(x_train), len(C2K_dot), len(C2K_dot[0])))
C2K_val = np.zeros((len(x_val), len(C2K_dot), len(C2K_dot[0])))
C2K_test = np.zeros((len(x_test), len(C2K_dot), len(C2K_dot[0])))

for i in range(len(x_train)):
    C2K_train[i] = C2K_dot

for i in range(len(x_val)):
    C2K_train[i] = C2K_dot

for i in range(len(x_test)):
    C2K_train[i] = C2K_dot

**Load Coarse Weights**

In [ ]:
for layer in base_model.layers:
    layer_name = layer.get_config()['name']
    if 'indep_layer' in layer_name:
        seg = layer_name.split('__')[1]
        for f_layer in model.layers:
            if seg in f_layer.get_config()['name']:
                f_layer.set_weights(layer.get_weights())

**Fit Training Data**

In [ ]:
index= 0
step = 1
stop = 1

while index < stop:
    model.fit([x_train, C2K_train], y_train, initial_epoch=index, epochs=index+step, batch_size=256, \
             validation_data=([x_val, C2K_val],y_val))
    model.save_weights('data/models/layer_'+str(index))
    index += step

**Evaluate on testing set**

In [ ]:
ct = np.zeros((20,len(x_test),100))
for i in range(len(x_test)):
    for j in range(20):
        ct[j,i] = C2K_dot[j]
        
model.evaluate([x_test, C2K_test], y_test, batch_size=64, verbose=1)